In [1]:
import numpy as np
from constants import *
from util import extract_results
from keras.models import load_model
from ultralytics import YOLO

In [2]:
NUM_VIDEOS = 10

In [3]:
detection_model = YOLO('../../data/detection_model/weights/last.pt')

In [4]:
action_model_v1 = load_model('../../data/action_model/v1.h5')
action_model_v2 = load_model('../../data/action_model/v2.h5')
action_model_v3 = load_model('../../data/action_model/v3.h5')
action_models = [action_model_v1, action_model_v2, action_model_v3]

In [5]:
def run(detection_model, action_model, model_ver):
    total_correct = 0
    total_count = 0

    for action in ACTIONS[:4] if model_ver == 1 else ACTIONS:
        correct = 0
        count = 0
        avg_confidence = 0
        for i in range (0, NUM_VIDEOS):
            sequence = []
            for j in range(0, 10):
                sensor_file = open(f"C:\\Users\\jeuch\\Documents\\GitHub\\traffic-watch\\data\\test\\%s\\%s\\sensors\\%s.txt" % (action,i,j))
                image_path = f"C:\\Users\\jeuch\\Documents\\GitHub\\traffic-watch\\data\\test\\%s\\%s\\videos\\%s.png" % (action,i,j)
                red_light_sensor = np.array([int(sensor_file.readline().split()[0])])
                results = detection_model(image_path, verbose=False)

                cropped_img_norm, keypoints_norm = extract_results(results, image_path=image_path)
                res = np.concatenate([cropped_img_norm, keypoints_norm[:2] if model_ver == 3 else keypoints_norm, red_light_sensor])
                sequence.append(res)
                sensor_file.close()

            res = action_model.predict(np.expand_dims(sequence, axis=0), verbose=None)[0]
            action_index = np.argmax(res)
            action_prediction = ACTIONS[action_index]
            #print(f"\t\t%s (%s%%)" % (action_prediction, int(res[action_index] * 100)))

            if action_prediction == action:
                correct += 1
            count += 1
            avg_confidence += res[action_index]
        total_correct += correct
        total_count += count
        print(f"\t%s: %s/%s (%s%%), avg confidence: %s%%" % (action, correct, count, int(correct/count * 100), int(avg_confidence/count * 100)))

    print(f"\n\tSummary: %s/%s (%s%%)\n" % (total_correct, total_count, int(total_correct/total_count * 100)))

In [23]:
model_version_temp = 1
action_model_temp = load_model('../../data/action_model/v%s_temp.h5' % model_version_temp)
run(detection_model, action_model_temp, model_version_temp)

	good: 2/10 (20%), avg confidence: 59%
	weaving: 6/10 (60%), avg confidence: 58%
	red_light: 10/10 (100%), avg confidence: 85%
	cross_yellow: 4/10 (40%), avg confidence: 53%

	Summary: 22/40 (55%)



In [9]:
for index, action_model in enumerate(action_models, start=1):
    print(f"v%s:" % index)
    run(detection_model, action_model, index)

v1:
	good: 1/5 (20%)
	weaving: 2/5 (40%)
	red_light: 4/5 (80%)
	cross_yellow: 2/5 (40%)

	Summary: 9/20 (45%)

v2:
	good: 0/5 (0%)
	weaving: 0/5 (0%)
	red_light: 3/5 (60%)
	cross_yellow: 3/5 (60%)
	off_road: 3/5 (60%)


KeyboardInterrupt: 